* Student Name: Daniel Mwaka
* Student Pace: DSF-FT12-Hybrid
* Instructor Name: Samuel Karu

# Box Office Performance Analysis for New Movie Studio

## Introduction 

The ever-increasing adoption and embracement of internet-hosted, media-sharing platforms exposes audiences to a diverse, highly-dense entertainment alternatives. This claim is justified by the rising number of companies entering the video streaming sector. Additionally, long-video content is increasigly facing stiff competition from short-video based content from social media sites such as Tiktok. Although venturing into the movie production sector is a potentially profitable portfolio diversification strategy; data-driven decision making is vital in orienting the company toward producing captivating, engaging, and appealing films to stratified target market segments. This project examines these factors systematically using a data-driven approach. 

# Problem Statement

The company plans to diversify its portfolio by launching a new division for movie production. Designing, implementing, sourcing talent, and operational expenses for running a new studio is a costly endevour. To ensure that the produces profitable movies, the company seeks data-driven insights to support appropriate corporate decisions.

## Analysis Focus

The project investigates the correlation  between runtime minutes, genre, and () on the grossing of films in the market.

# Objectives

<strong> 1: Understanding the Dataset </strong>

* <strong> Goal: </strong> Gain an indepth understanding on the datsets.

* <strong> Tasks: </strong>
    * Review shape, columns, data types.
    * Dropping unnecessary columns/ fields 
    * Data cleaning (remove duplicates and handle missing values) 

<strong> 2: Industry Background </strong>

* <strong> Goal: </strong> Comprehend trends in the film industry and triangulate potential predictor variables for a film's total grossing.  

* <strong> Tasks: </strong>
    
    *
    
    *

In [26]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
%matplotlib inline


In [7]:
# Load the data from the .csv file as a DataFrame and display first five rows
bom_df = pd.read_csv('../zipped-data/bom.movie_gross.csv')
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [8]:
# Connect to the IMDB database
conn = sqlite3.connect('../zipped-data/im.db')

In [9]:
# Load the movie_basics table and display first five rows
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [10]:
# Load the movie-ratings table and print first-five rows
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [11]:
# Load the directors table and print first-five rows
directors = pd.read_sql_query("SELECT * FROM directors", conn)
directors.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [12]:
# Load the writers table and print first-five rows
writers = pd.read_sql_query("SELECT * FROM writers", conn)
writers.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


In [13]:
# Load the persons table and display first-five rows
persons = pd.read_sql_query("SELECT * FROM persons", conn)
persons.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [14]:
# Define a function to check for missing values and duplicates for the dataframe passed as an argument
def check_duplicates_missing(df):
    print('Duplicated:')
    print(df.duplicated().value_counts())
    print('\nMissing values:')
    print(df.isna().sum())
    return df

Apply the check_duplicate_missing function for each of the 6 dataframes

In [18]:
# Run the check on bom_df
print("BOM DataFrame:")
check_duplicates_missing(bom_df)

# Run the check on movie_basics
print("\nMovie Basics DataFrame:")
check_duplicates_missing(movie_basics)

# Run the check on movie_ratings
print("\nMovie Ratings DataFrame:")
check_duplicates_missing(movie_ratings)

# Run the check on directors
print("\nDirectors DataFrame:")
check_duplicates_missing(directors)

# Run the check on writers
print("\nWriters DataFrame:")
check_duplicates_missing(writers)

# Run the check on persons
print("\nPersons DataFrame:")
check_duplicates_missing(persons)

BOM DataFrame:
Duplicated:
False    3387
dtype: int64

Missing values:
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

Movie Basics DataFrame:
Duplicated:
False    146144
dtype: int64

Missing values:
movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

Movie Ratings DataFrame:
Duplicated:
False    73856
dtype: int64

Missing values:
movie_id         0
averagerating    0
numvotes         0
dtype: int64

Directors DataFrame:
Duplicated:
False    163535
True     127639
dtype: int64

Missing values:
movie_id     0
person_id    0
dtype: int64

Writers DataFrame:
Duplicated:
False    178352
True      77521
dtype: int64

Missing values:
movie_id     0
person_id    0
dtype: int64

Persons DataFrame:
Duplicated:
False    606648
dtype: int64

Missing values:
person_id                  0
primary_name       

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


Defining a function for handling missing values. The criteria for handling missing values is as follows:
* Drop columns with more than 50% missing values.
* Impute with mean if distribuction is fairly symetrical (low skewness).
* Impute with median if distribuction is skewed. 
* Impute with mode for categorical columns (if mode not available fill with Unknown).

In [19]:
# Define a function to handle missing values and duplicates
def filling_missing(df):
    # Dropping duplicate rows
    df = df.drop_duplicates()

    # Dropping columns with more than 50% missing values
    df = df.dropna(thresh=0.3 * len(df), axis=1)


    # Filling missing values in numeric columns
    for col in df.select_dtypes(include=['number']).columns:
        if df[col].isna().sum() > 0:
            skewness = df[col].skew()
            fill_value = df[col].mean() if abs(skewness) < 0.5 else df[col].median()
            df[col].fillna(fill_value, inplace=True)
    
    # Filling missing values in categorical columns
    for col in df.select_dtypes(exclude=['number']).columns:
        if df[col].isna().sum() > 0:
            most_frequent = df[col].mode()[0] if not df[col].mode().empty else "Unknown"
            df[col].fillna(most_frequent, inplace=True)

    return df

In [20]:
# Call the filling_missing function and pass each of the 6 DataFrames to drop duplicates and impute missing values
bom_df = filling_missing(bom_df)
movie_basics = filling_missing(movie_basics)
movie_ratings = filling_missing(movie_ratings)
directors = filling_missing(directors)
writers = filling_missing(writers)
persons = filling_missing(persons)

In [21]:
# Convert gross columns to numeric
bom_df['domestic_gross'] = pd.to_numeric(bom_df['domestic_gross'], errors='coerce')
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')

In [22]:
# Verify the cleaned DataFrames neither have missing values nor duplicates

# Run the check on bom_df
print("BOM DataFrame:")
check_duplicates_missing(bom_df)

# Run the check on movie_basics
print("\nMovie Basics DataFrame:")
check_duplicates_missing(movie_basics)

# Run the check on movie_ratings
print("\nMovie Ratings DataFrame:")
check_duplicates_missing(movie_ratings)

# Run the check on directors
print("\nDirectors DataFrame:")
check_duplicates_missing(directors)

# Run the check on writers
print("\nWriters DataFrame:")
check_duplicates_missing(writers)

# Run the check on persons
print("\nPersons DataFrame:")
check_duplicates_missing(persons)

BOM DataFrame:
Duplicated:
False    3387
dtype: int64

Missing values:
title             0
studio            0
domestic_gross    0
foreign_gross     5
year              0
dtype: int64

Movie Basics DataFrame:
Duplicated:
False    146144
dtype: int64

Missing values:
movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

Movie Ratings DataFrame:
Duplicated:
False    73856
dtype: int64

Missing values:
movie_id         0
averagerating    0
numvotes         0
dtype: int64

Directors DataFrame:
Duplicated:
False    163535
dtype: int64

Missing values:
movie_id     0
person_id    0
dtype: int64

Writers DataFrame:
Duplicated:
False    178352
dtype: int64

Missing values:
movie_id     0
person_id    0
dtype: int64

Persons DataFrame:
Duplicated:
False    606648
dtype: int64

Missing values:
person_id             0
primary_name          0
primary_profession    0
dtype: int64


,person_id,primary_name,primary_profession
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,"production_designer,art_department,set_decorator"
...,...,...,...
606643,nm9990381,Susan Grobes,actress
606644,nm9990690,Joo Yeon So,actress
606645,nm9991320,Madeline Smith,actress
606646,nm9991786,Michelle Modigliani,producer


Define a function based on the Interquartile Range (IQR) method to remove outliers from columns

In [23]:
# remove outliers using IQR
def remove_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

    
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

Call the remove_outliers() function to remove outliers from numerical columns across the movie_ratings, bom_df, movie_basics, and movie_ratings DataFrames

In [24]:
movie_ratings = remove_outliers(movie_ratings, ['averagerating', 'numvotes'])
bom_df = remove_outliers(bom_df, ['domestic_gross', 'foreign_gross'])
movie_basics = remove_outliers(movie_basics, ['runtime_minutes'])
movie_ratings = remove_outliers(movie_ratings, ['averagerating', 'numvotes'])

<strong> Merging DataFrames and Final Data Cleaning </strong>
  
<strong> 1: Merging DataFrames </strong>

* Merge `bom_df` with `movie_basics` based on the `title` and `primary_title` columns.

* Merge the merged DataFrame with `movie_ratings`, `directors`, and `writers` based on the `movie_id` column.
 
<strong> 2: Handling Duplicates </strong>

* Any duplicates in the resulting `final_df` are dropped by calling the drop_duplicates() function that was defined earlier to ensure the dataframe used from analysis doesn't include duplicate entries.

<strong> 3: Filling Missing Values </strong>

* The code also features an option to fill missing value with `Unknown` but the step is commented out in the current execution.


<strong> 4: Final Check </strong>

* The .info() method is called to inspect the final version of the cleaned DataFrame and justify the data is ready for analysis.

In [25]:
# Merge BOM DataFrame with Movie Basics DataFrame based on title and primary_title
merged_df = pd.merge(bom_df, movie_basics, left_on='title', right_on='primary_title', how='inner')

# merging with movie_ratings
final_df = pd.merge(merged_df, movie_ratings, on='movie_id', how='inner')

# Merging with directors
final_df = pd.merge(final_df, directors, on='movie_id', how='left')

# Merging with writers
final_df = pd.merge(final_df, writers, on='movie_id', how='left')

# Drop duplicates
final_df = final_df.drop_duplicates()

# Fill missing values
#final_df.fillna('Unknown', inplace=True)

# Final check
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 682
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            683 non-null    object 
 1   studio           683 non-null    object 
 2   domestic_gross   683 non-null    float64
 3   foreign_gross    683 non-null    float64
 4   year             683 non-null    int64  
 5   movie_id         683 non-null    object 
 6   primary_title    683 non-null    object 
 7   original_title   683 non-null    object 
 8   start_year       683 non-null    int64  
 9   runtime_minutes  683 non-null    float64
 10  genres           683 non-null    object 
 11  averagerating    683 non-null    float64
 12  numvotes         683 non-null    int64  
 13  person_id_x      681 non-null    object 
 14  person_id_y      641 non-null    object 
dtypes: float64(4), int64(3), object(8)
memory usage: 85.4+ KB
